In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.densenet import DenseNet121,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexEntropy"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = DenseNet121(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1, 1, 1024)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1024)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   52.6s


[CV] ........................... C=0.01, score=0.927435, total=  51.9s
[CV] ........................... C=0.01, score=0.913690, total=  52.0s
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.923228, total=  52.8s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.906863, total=  53.2s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.927794, total=  54.0s
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.905512, total=  54.9s
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.919765, total=  55.7s
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.5min


[CV] ............................ C=0.1, score=0.930762, total= 2.5min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.919765, total= 2.6min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.913690, total= 2.6min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.912745, total= 2.6min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.927651, total= 2.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.918627, total= 2.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.911417, total= 2.7min
[CV] ............................ C=0.1, score=0.911355, total= 2.7min
[CV] C

...

[CV] ................... C=1000000000.0, score=0.898438, total= 2.5min
[CV] ................... C=1000000000.0, score=0.908824, total= 2.5min
[CV] ................... C=1000000000.0, score=0.906496, total= 2.5min
[CV] ................... C=1000000000.0, score=0.915925, total= 2.4min
[CV] ................... C=1000000000.0, score=0.915507, total= 2.3min
[CV] ................... C=1000000000.0, score=0.906375, total= 2.2min
[CV] ................... C=1000000000.0, score=0.903543, total= 2.8min
[CV] ................... C=1000000000.0, score=0.901786, total= 2.6min
[CV] ................... C=1000000000.0, score=0.913776, total= 2.7min
[CV] .................. C=10000000000.0, score=0.898438, total= 2.5min
[CV] .................. C=10000000000.0, score=0.903543, total= 2.0min
[CV] .................. C=10000000000.0, score=0.901174, total= 2.5min
[CV] .................. C=10000000000.0, score=0.902077, total= 1.9min
[CV] .................. C=10000000000.0, score=0.908824, total= 2.3min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 14.1min remaining:   33.7s


[CV] .................. C=10000000000.0, score=0.916749, total= 2.0min
[CV] .................. C=10000000000.0, score=0.913386, total= 2.2min
[CV] .................. C=10000000000.0, score=0.906375, total= 1.8min
[CV] .................. C=10000000000.0, score=0.909722, total= 2.0min
[CV] .................. C=10000000000.0, score=0.915507, total= 2.0min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 14.3min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 53.49081681, 157.19331725, 160.38788171, 156.29761982,
        162.72076263, 166.21642797, 160.93570716, 165.44960842,
        160.61575391, 163.87164097, 165.39915836, 149.41567287,
        128.3047972 ]),
 'mean_score_time': array([0.03009624, 0.0267894 , 0.0237344 , 0.02026265, 0.02222235,
        0.02276561, 0.02111402, 0.02395391, 0.01949399, 0.01974652,
        0.02025862, 0.01339529, 0.01028054]),
 'mean_test_score': array([0.91663378, 0.92048145, 0.9132794 , 0.90982636, 0.90518942,
        0.90578137, 0.90962904, 0.90864246, 0.90814917, 0.90933307,
        0.90272297, 0.90716259, 0.90755722]),
 'mean_train_score': array([0.95427736, 0.98231853, 0.98902755, 0.98481639, 0.9841924 ,
        0.98469657, 0.98651586, 0.98707559, 0.9865704 , 0.9866691 ,
        0.98369969, 0.98591394, 0.98761343]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.91663, std: 0.00914, params: {'C': 0.01},
 mean: 0.92048, std: 0.00729, params: {'C': 0.1},
 mean: 0.91328, std: 0.00611, params: {'C': 1.0},
 mean: 0.90983, std: 0.00830, params: {'C': 10.0},
 mean: 0.90519, std: 0.00617, params: {'C': 100.0},
 mean: 0.90578, std: 0.00672, params: {'C': 1000.0},
 mean: 0.90963, std: 0.00825, params: {'C': 10000.0},
 mean: 0.90864, std: 0.00872, params: {'C': 100000.0},
 mean: 0.90815, std: 0.00659, params: {'C': 1000000.0},
 mean: 0.90933, std: 0.00785, params: {'C': 10000000.0},
 mean: 0.90272, std: 0.00720, params: {'C': 100000000.0},
 mean: 0.90716, std: 0.00590, params: {'C': 1000000000.0},
 mean: 0.90756, std: 0.00600, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.9205
